# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

## Data

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
import os

data_dir = os.getcwd()
data = pd.read_csv(os.path.join(data_dir, "assignment3_creditcard.csv"), encoding='utf8')

data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.848212,2.384900,0.379573,1.048381,-0.845070,2.537837,-4.542983,-10.201458,-1.504967,-2.234167,...,2.585817,-5.291690,0.859364,0.423231,-0.506985,1.020052,-0.627751,-0.017753,0.280982,0
1,2.071805,-0.477943,-1.444444,-0.548657,0.010036,-0.582242,-0.042878,-0.247160,1.171923,-0.342382,...,-0.077306,0.042858,0.390125,0.041569,0.598427,0.098803,0.979686,-0.093244,-0.065615,0
2,-2.985294,-2.747472,1.194068,-0.003036,-1.151041,-0.263559,0.553500,0.635600,0.438545,-1.806488,...,1.345776,0.373760,-0.385777,1.197596,0.407229,0.008013,0.762362,-0.299024,-0.303929,0
3,-1.479452,1.542874,0.290895,0.838142,-0.529290,-0.717661,0.484516,0.545092,-0.780767,0.324804,...,0.038397,0.116771,0.405560,-0.116453,0.541275,-0.216665,-0.415578,0.027126,-0.150347,0
4,-0.281976,-0.309699,-2.162299,-0.851514,0.106167,-1.483888,1.930994,-0.843049,-1.249272,1.079608,...,-0.875516,-0.004199,1.015108,-0.026748,0.077115,-1.468822,0.751700,0.496732,0.331001,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28678 entries, 0 to 28677
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      28678 non-null  float64
 1   V2      28678 non-null  float64
 2   V3      28678 non-null  float64
 3   V4      28678 non-null  float64
 4   V5      28678 non-null  float64
 5   V6      28678 non-null  float64
 6   V7      28678 non-null  float64
 7   V8      28678 non-null  float64
 8   V9      28678 non-null  float64
 9   V10     28678 non-null  float64
 10  V11     28678 non-null  float64
 11  V12     28678 non-null  float64
 12  V13     28678 non-null  float64
 13  V14     28678 non-null  float64
 14  V15     28678 non-null  float64
 15  V16     28678 non-null  float64
 16  V17     28678 non-null  float64
 17  V18     28678 non-null  float64
 18  V19     28678 non-null  float64
 19  V20     28678 non-null  float64
 20  V21     28678 non-null  float64
 21  V22     28678 non-null  float64
 22

In [4]:
data.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
count,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,...,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000
mean,-0.015438,0.053653,-0.046031,0.037348,-0.033724,-0.003299,-0.051054,0.006064,-0.018530,-0.041149,...,0.002633,0.010289,-0.000656,-0.004800,-0.000897,-0.001989,-0.000765,0.000948,0.001535,0.008578
std,2.031529,1.616186,1.758169,1.482109,1.486998,1.339259,1.454827,1.364342,1.134065,1.252593,...,0.720307,0.847152,0.739469,0.593663,0.603349,0.517968,0.483852,0.397075,0.296736,0.092221
min,-30.552380,-42.172688,-31.103685,-5.560118,-42.147898,-21.929312,-41.506796,-39.267378,-13.434066,-24.403185,...,-21.387122,-21.453736,-8.887017,-36.666000,-2.718024,-6.712624,-2.241620,-7.418878,-9.617915,0.000000
25%,-0.916927,-0.575381,-0.899872,-0.843321,-0.714901,-0.763757,-0.568146,-0.206103,-0.661909,-0.543450,...,-0.209678,-0.225520,-0.539244,-0.160583,-0.356047,-0.318619,-0.327343,-0.070558,-0.052189,0.000000
50%,0.020050,0.075358,0.180610,-0.008844,-0.060040,-0.271363,0.036107,0.022463,-0.055095,-0.097390,...,-0.062792,-0.028778,0.007302,-0.011199,0.040006,0.019770,-0.056260,0.002049,0.011075,0.000000
75%,1.317461,0.806957,1.029928,0.771958,0.613328,0.397269,0.559409,0.329606,0.605704,0.460681,...,0.131199,0.184312,0.526358,0.146835,0.437146,0.352717,0.240713,0.091637,0.078911,0.000000
max,2.399484,21.467203,4.069865,11.927512,32.911462,22.529298,36.677268,20.007208,8.113152,15.236028,...,26.237391,27.202839,8.361985,9.637187,3.948061,2.510401,3.122747,11.135740,14.929133,1.000000


In [5]:
# 결측치 확인 -> 없음

data.isnull().sum().sum()

0

In [6]:
X = data.drop('Class', axis=1)
y = data['Class']


### 정규화

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))

columns = X.columns

X = scaler.fit_transform(X)

X = pd.DataFrame(X, columns=columns)

X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,0.742188,0.400304,0.790167,-0.244209,0.100538,0.100671,-0.054441,-0.019279,0.107252,0.118540,...,-0.029430,0.006748,-0.335669,0.130081,0.602016,-0.336630,0.676820,-0.398300,-0.202234,-0.193476
1,0.980112,0.310334,0.686451,-0.426856,0.123323,-0.039688,0.060674,0.316592,0.355719,0.213990,...,0.069118,-0.105090,-0.116395,0.075673,0.585531,-0.004978,0.477048,0.201001,-0.210371,-0.221715
2,0.673173,0.239010,0.836480,-0.364456,0.092385,-0.025352,0.075930,0.346377,0.287648,0.140118,...,-0.241714,-0.045328,-0.102794,-0.014292,0.635464,-0.062342,0.457361,0.119976,-0.232552,-0.241132
3,0.764570,0.373842,0.785124,-0.268253,0.108952,-0.045780,0.074166,0.343323,0.174472,0.247653,...,0.072861,-0.100231,-0.113357,0.077463,0.578705,-0.022125,0.408640,-0.319196,-0.197396,-0.228619
4,0.837250,0.315621,0.645634,-0.461493,0.125884,-0.080249,0.111167,0.296486,0.130985,0.285737,...,-0.169881,-0.138611,-0.118329,0.148139,0.582580,-0.161385,0.137111,0.116001,-0.146777,-0.189400


### Train, Test Split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [9]:
y.value_counts()

0    28432
1      246
Name: Class, dtype: int64

y의 분포가 매우 불균형 한 것을 볼 수 있다.

Accuracy의 경우, 분포가 편향되어 있는 경우에 성능 평가 지표로 사용하기에 무리가 있다. 

<h1> Logistic Regression 분석 </h1>

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test) 

<h1> 성능지표 계산 </h1>

In [11]:
from sklearn.metrics import confusion_matrix, roc_auc_score

- confusion matrix

In [12]:
cm = confusion_matrix(y_test, pred)

TP = cm[1, 1]
FP = cm[0, 1]
FN = cm[1, 0]
TN = cm[0, 0]

- 정확도 (Accuracy)

In [13]:
accuracy = (TP + TN) / (TP + FP + FN + TN)
accuracy

0.9961645746164575

- 정밀도 (Precision) : 모델이 Positive로 예측한 케이스 중 실제로 Positive인 케이스의 비율

In [14]:
precision = TP / (TP + FP)
precision

1.0

- 재현율 (Recall) : 실제로 Positive인 케이스 중 모델이 Positive로 예측한 케이스의 비율

In [15]:
recall = TP / (TP + FN)
recall

0.6764705882352942

- F1 Score : 정밀도와 재현율의 조화 평균

In [16]:
f1 = 2 * precision * recall / (precision + recall)
f1

0.8070175438596492

- AUC-ROC Curve : 모델이 Positive 클래스와 Negative 클래스를 구분하는 능력을 평가하는 지표

In [17]:
y_score = model.predict_proba(X_test)[:,1] 

roc_auc = roc_auc_score(y_test, y_score)
roc_auc

0.9672360828593963

<h1> 성능 개선 시도 : 임계값(cutoff) 조정 </h1>

기본적으로 로지스틱 회귀 모델은 예측 확률이 0.5 이상인 경우에만 해당 케이스를 Positive로 분류한다. 하지만 이 임계값을 조정함으로써 모델의 정밀도와 재현율 사이의 균형을 조절할 수 있다. 

In [18]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

precision_new, recall_new, thresholds = precision_recall_curve(y_test, y_score)

f1_scores = 2*recall_new*precision_new / (recall_new + precision_new)

optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

print('Optimal cutoff : ', optimal_threshold)


Optimal cutoff :  0.20595051256436353


In [19]:
# optimal_threshold을 사용하여 예측 수행

pred_optimal = np.where(y_score >= optimal_threshold, 1, 0)

### 각 성능 지표 다시 계산

In [20]:
cm_new = confusion_matrix(y_test, pred_optimal)

TP_new = cm_new[1, 1]
FP_new = cm_new[0, 1]
FN_new = cm_new[1, 0]
TN_new = cm_new[0, 0]

In [21]:
accuracy_new = (TP_new + TN_new) / (TP_new + FP_new + FN_new + TN_new)
precision_new = TP_new / (TP_new + FP_new)
recall_new = TP_new / (TP_new + FN_new)
f1_new = 2 * precision_new * recall_new / (precision_new + recall_new)
roc_auc_new = roc_auc_score(y_test, y_score)


In [22]:
import pandas as pd

Metrics = {'Metrics': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC'],
           'Old Value': [accuracy, precision, recall, f1, roc_auc],
           'New Value': [accuracy_new, precision_new, recall_new, f1_new, roc_auc_new]}

pd.DataFrame(Metrics)


,Metrics,Old Value,New Value
0,Accuracy,0.996165,0.997211
1,Precision,1.000000,1.000000
2,Recall,0.676471,0.764706
3,F1 Score,0.807018,0.866667
4,ROC-AUC,0.967236,0.967236
